In [1]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

In [2]:
class MLP_500_500_500_regressor(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Linear(22, 500)  
        self.relu = nn.ReLU()
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU()
        )                          
        self.layer2 = nn.Linear(500, 30) 

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [3]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()
y_regression = y.astype(np.float32)  # Garanta que é float32

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [4]:
print(y)

[[0.7        0.5        0.72857143 ... 0.71428571 0.72857143 0.71428571]
 [0.72857143 0.5        0.7        ... 0.68571429 0.7        0.71428571]
 [0.75714286 0.5        0.75714286 ... 0.75714286 0.77142857 0.77142857]
 ...
 [0.52285714 0.5        0.50857143 ... 0.50571429 0.50571429 0.50857143]
 [0.5        0.5        0.49285714 ... 0.50428571 0.49857143 0.5       ]
 [0.51714286 0.5        0.52428571 ... 0.49       0.52142857 0.50428571]]


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [6]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [7]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000):
    model.to(device)
    early_stopping = EarlyStopping(patience=10, min_delta=0.0)
    best_model = None
    best_val_loss = 1000000
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0

            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()
        
        
        val_loss /= len(val_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} ')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_val_loss

In [8]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_regression[train_idx], y_regression[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_500_500_500_regressor()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization
    
    # Treinar fold
    best_model, best_mse = train_fold(model, train_loader, val_loader, criterion, optimizer)
    results.append(best_mse)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-500-500-500-regressor_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f" MSE médio: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"MSE por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 1.6558 - Val Loss: 0.0335 
Epoch 2/1000000 - Train Loss: 0.3417 - Val Loss: 0.0314 
Epoch 3/1000000 - Train Loss: 0.1296 - Val Loss: 0.0111 
Epoch 4/1000000 - Train Loss: 0.0729 - Val Loss: 0.0069 
Epoch 5/1000000 - Train Loss: 0.0491 - Val Loss: 0.0047 
Epoch 6/1000000 - Train Loss: 0.0347 - Val Loss: 0.0034 
Epoch 7/1000000 - Train Loss: 0.0225 - Val Loss: 0.0019 
Epoch 8/1000000 - Train Loss: 0.0138 - Val Loss: 0.0017 
Epoch 9/1000000 - Train Loss: 0.0119 - Val Loss: 0.0015 
Epoch 10/1000000 - Train Loss: 0.0114 - Val Loss: 0.0014 
Epoch 11/1000000 - Train Loss: 0.0109 - Val Loss: 0.0014 
Epoch 12/1000000 - Train Loss: 0.0108 - Val Loss: 0.0014 
Epoch 13/1000000 - Train Loss: 0.0110 - Val Loss: 0.0014 
Epoch 14/1000000 - Train Loss: 0.0104 - Val Loss: 0.0013 
Epoch 15/1000000 - Train Loss: 0.0106 - Val Loss: 0.0013 
Epoch 16/1000000 - Train Loss: 0.0105 - Val Loss: 0.0013 
Epoch 17/1000000 - Train Loss: 0.0106 - Val Loss: 0.0014 
Epoch